In [ ]:
import os
from PIL import Image

def trova_immagine_con_risoluzione_minima(cartelle):
    x = None
    y = None
    min = 1000
    count = 0

    for cartella in cartelle:
        for file in os.listdir(cartella):
            if file.endswith('.jpg') or file.endswith('.png'):
                img = Image.open(os.path.join(cartella, file))
                x=img.size[0]
                y=img.size[1]
                if x < 240 or y < 240:
                    #os.remove(os.path.join(cartella, file))
                    count = count +1
                if min > x or min > y:
                    min = x if x < y else y
                    min_file = os.path.join(cartella, file)
    print(f"Numero di immagini controllate: {count}")
    return min_file, min

cartelle = ['videos/Fire', 'videos/NoFire']
file, min = trova_immagine_con_risoluzione_minima(cartelle)
print(f"L'immagine con la risoluzione più bassa è {file} con una risoluzione di {min}")

In [9]:
import os
import cv2
import numpy as np
from tqdm import tqdm


DATADIR = 'videos'
CATEGORIES = ['Fire', 'NoFire']

IMG_SIZE = 240
def create_training_data():
    training_data = []
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
              
    return training_data


In [10]:
training_data = create_training_data()

100%|██████████| 1301/1301 [00:14<00:00, 88.31it/s] 


In [11]:
import random

print(len(training_data))
random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

2423
1
0
1
0
1
0
1
0
1
0


In [12]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0
X.shape[1:]
Y = np.array(Y)

In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

model = Sequential()


model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=2, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
history = model.fit(X, Y, batch_size=32, epochs=100,validation_split=0.3)

Epoch 1/100
53/53 [==============================] - 4s 49ms/step - loss: 0.7378 - accuracy: 0.6309 - val_loss: 0.5398 - val_accuracy: 0.7290
Epoch 2/100
53/53 [==============================] - 2s 31ms/step - loss: 0.5154 - accuracy: 0.7571 - val_loss: 0.4682 - val_accuracy: 0.7730
Epoch 3/100
53/53 [==============================] - 2s 31ms/step - loss: 0.4553 - accuracy: 0.7954 - val_loss: 0.4127 - val_accuracy: 0.8391
Epoch 4/100
53/53 [==============================] - 2s 31ms/step - loss: 0.4160 - accuracy: 0.8066 - val_loss: 0.4542 - val_accuracy: 0.7840
Epoch 5/100
53/53 [==============================] - 2s 31ms/step - loss: 0.3814 - accuracy: 0.8379 - val_loss: 0.3676 - val_accuracy: 0.8404
Epoch 6/100
53/53 [==============================] - 2s 30ms/step - loss: 0.3543 - accuracy: 0.8408 - val_loss: 0.3614 - val_accuracy: 0.8487
Epoch 7/100
53/53 [==============================] - 2s 30ms/step - loss: 0.3288 - accuracy: 0.8473 - val_loss: 0.5271 - val_accuracy: 0.7730
Epoch 

In [15]:
hdf5_model_path = 'fire_detection.h5'
model.save(hdf5_model_path, save_format='h5')